# Cleanup Lab Resources

This notebook helps you clean up all resources created in Labs 1-3:
- Runtime agents (QA Agent, Supervisor Agent)
- Memory resources
- SSM parameters

Run this when you want to start fresh or remove all lab resources.

## Step 1: Setup and Credentials

In [ ]:
import boto3
import os
from bedrock_agentcore.memory import MemoryClient

print("✅ Libraries imported")

In [ ]:
# Set AWS credentials - Update these with your temporary credentials
os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_ACCESS_KEY'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_SECRET_KEY'
os.environ['AWS_SESSION_TOKEN'] = 'YOUR_SESSION_TOKEN'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

# Clear boto3 cached sessions
boto3.DEFAULT_SESSION = None

# Verify credentials
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"✅ Using AWS Account: {identity['Account']}")
print(f"✅ User/Role: {identity['Arn']}")

## Step 2: Helper Functions

In [ ]:
def get_ssm_parameter(parameter_name: str):
    """Get parameter value from SSM Parameter Store."""
    ssm = boto3.client('ssm')
    try:
        response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)
        return response['Parameter']['Value']
    except Exception as e:
        print(f"⚠️  Parameter {parameter_name} not found: {e}")
        return None

def delete_ssm_parameter(parameter_name: str):
    """Delete SSM parameter."""
    ssm = boto3.client('ssm')
    try:
        ssm.delete_parameter(Name=parameter_name)
        print(f"✅ Deleted SSM parameter: {parameter_name}")
        return True
    except Exception as e:
        print(f"⚠️  Could not delete {parameter_name}: {e}")
        return False

def delete_runtime_agent(agent_arn: str):
    """Delete runtime agent."""
    client = boto3.client('bedrock-agentcore', region_name='us-west-2')
    try:
        client.delete_agent_runtime(agentRuntimeArn=agent_arn)
        print(f"✅ Deleted runtime agent: {agent_arn}")
        return True
    except Exception as e:
        print(f"⚠️  Could not delete agent {agent_arn}: {e}")
        return False

def delete_memory_resource(memory_id: str):
    """Delete memory resource."""
    memory_client = MemoryClient(region_name='us-west-2')
    try:
        memory_client.delete_memory(memory_id=memory_id)
        print(f"✅ Deleted memory resource: {memory_id}")
        return True
    except Exception as e:
        print(f"⚠️  Could not delete memory {memory_id}: {e}")
        return False

print("✅ Helper functions defined")

## Step 3: List Current Resources

First, let's see what resources exist.

In [ ]:
print("📋 Current Lab Resources:")
print("=" * 60)

# Check QA Agent
qa_agent_arn = get_ssm_parameter("/app/investment/agentcore/qa_agent_arn")
if qa_agent_arn:
    print(f"\n✓ QA Agent: {qa_agent_arn}")
else:
    print("\n✗ QA Agent: Not found")

# Check Supervisor Agent
supervisor_agent_arn = get_ssm_parameter("/app/investment/agentcore/supervisor_agent_arn")
if supervisor_agent_arn:
    print(f"✓ Supervisor Agent: {supervisor_agent_arn}")
else:
    print("✗ Supervisor Agent: Not found")

# Check Memory Resource
memory_id = get_ssm_parameter("/app/investment/agentcore/memory_id")
if memory_id:
    print(f"✓ Memory Resource: {memory_id}")
else:
    print("✗ Memory Resource: Not found")

print("\n" + "=" * 60)

## Step 4: Delete QA Agent

Delete the Quantitative Analysis runtime agent.

In [ ]:
print("🗑️  Deleting QA Agent...")
print("=" * 60)

if qa_agent_arn:
    delete_runtime_agent(qa_agent_arn)
    delete_ssm_parameter("/app/investment/agentcore/qa_agent_arn")
else:
    print("⚠️  No QA Agent to delete")

print("=" * 60)

## Step 5: Delete Supervisor Agent

Delete the Memory-Enhanced Supervisor runtime agent.

In [ ]:
print("🗑️  Deleting Supervisor Agent...")
print("=" * 60)

if supervisor_agent_arn:
    delete_runtime_agent(supervisor_agent_arn)
    delete_ssm_parameter("/app/investment/agentcore/supervisor_agent_arn")
else:
    print("⚠️  No Supervisor Agent to delete")

print("=" * 60)

## Step 6: Delete Memory Resource

Delete the memory resource and all stored memories.

In [ ]:
print("🗑️  Deleting Memory Resource...")
print("=" * 60)

if memory_id:
    delete_memory_resource(memory_id)
    delete_ssm_parameter("/app/investment/agentcore/memory_id")
else:
    print("⚠️  No Memory Resource to delete")

print("=" * 60)

## Step 7: Verify Cleanup

Verify all resources have been deleted.

In [ ]:
print("🔍 Verifying cleanup...")
print("=" * 60)

all_clean = True

# Check QA Agent
if get_ssm_parameter("/app/investment/agentcore/qa_agent_arn"):
    print("⚠️  QA Agent SSM parameter still exists")
    all_clean = False
else:
    print("✅ QA Agent cleaned up")

# Check Supervisor Agent
if get_ssm_parameter("/app/investment/agentcore/supervisor_agent_arn"):
    print("⚠️  Supervisor Agent SSM parameter still exists")
    all_clean = False
else:
    print("✅ Supervisor Agent cleaned up")

# Check Memory Resource
if get_ssm_parameter("/app/investment/agentcore/memory_id"):
    print("⚠️  Memory Resource SSM parameter still exists")
    all_clean = False
else:
    print("✅ Memory Resource cleaned up")

print("\n" + "=" * 60)
if all_clean:
    print("\n🎉 All resources cleaned up successfully!")
    print("\nYou can now start fresh with Lab 1.")
else:
    print("\n⚠️  Some resources may still exist. Check the AWS Console.")
    print("\nNote: Runtime agent deletion may take a few minutes to complete.")

## Cleanup Complete

All lab resources have been deleted. You can now:
- Start fresh with Lab 1
- Create new agents with different configurations
- Implement the custom memory strategy

### Manual Verification (Optional)

You can verify deletion in the AWS Console:
- **Runtime Agents**: Amazon Bedrock → AgentCore → Runtimes
- **Memory Resources**: Amazon Bedrock → AgentCore → Memory
- **SSM Parameters**: Systems Manager → Parameter Store

Or use AWS CLI:
```bash
# List runtime agents
aws bedrock-agentcore list-agent-runtimes --region us-west-2

# List memory resources
aws bedrock-agentcore list-memories --region us-west-2

# List SSM parameters
aws ssm get-parameters-by-path --path /app/investment/agentcore --region us-west-2
```